In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
%cd gdrive/My\ Drive/rubik_project

/content/gdrive/My Drive/rubik_project


In [4]:
!nvidia-smi

Sat Jan 18 10:46:41 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import os
import numpy as np
import cv2
import PIL
import imutils

In [6]:
os.listdir()

['data',
 '__pycache__',
 'face_detector',
 'data_utils.py',
 'face_multitask.jpg',
 'config.py',
 'data_provider.py',
 'model.py',
 'checkpoints',
 'bknet_4braches.ipynb',
 'test.ipynb']

**Evaluating trained model**

In [0]:
%tensorflow_version 1.x

In [8]:
import tensorflow as tf
print(tf.__version__)

1.15.0


In [9]:
SAVE_FOLDER = os.path.join(os.getcwd(), 'checkpoints')
print(SAVE_FOLDER)
print(os.listdir(SAVE_FOLDER))

/content/gdrive/My Drive/rubik_project/checkpoints
['17012020model.ckpt.data-00000-of-00001', '17012020model.ckpt.index', 'checkpoint', '17012020model.ckpt.meta', '17012020']


In [0]:
import data_provider
import config
import data_utils

In [11]:
test_data = data_provider.Dataset('public_test', config.BATCH_SIZE)

Load smile image...................
Done !
Number of smile train data:  3000
---------------------------------------------------------------
Load emotion image..................
Done !
Number of emotion train data:  28709
---------------------------------------------------------------
Load gender image...................
Done !
Number of gender train data:  30000
---------------------------------------------------------------
Load age image..................
Done !
Number of age train data:  27353
---------------------------------------------------------------


In [12]:
print(len(test_data.all_data))

15648


In [0]:
sess = tf.InteractiveSession()

In [16]:
MODEL_FOLDER = os.path.join(SAVE_FOLDER, '17012020')
ckpt = tf.train.get_checkpoint_state(MODEL_FOLDER)
print(ckpt.model_checkpoint_path)

/content/gdrive/My Drive/rubik_project/checkpoints/17012020/model.ckpt


In [0]:
import model

In [18]:
test_model = model.Multitask_BKNet(sess)




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

(?, 24, 24, 32)
(?, 12, 12, 64)
(?, 6, 6, 128)
(?, 3, 3, 256)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



Load smile image...................
Done !
Number of smile train data:  3000
---------------------------------------------------------------
Load emotion image..................
Done !
Number of emotion train data:  28709
---------------------------------------------------------------
Load gender image...................
Done !
Number of gender train data:  30000
---------------------------------------------------------------
Load age image..................
Done !
Number of age train data:  27353
---------------------------------------------------------------

Instructions for updating:
Please use tf.global_variables instead.
Reading model parameters from %s /content/gdrive/My Drive/rubik_project/checkpoints/17

In [19]:
smile_nb_true_pred = 0
emotion_nb_true_pred = 0
gender_nb_true_pred = 0
age_nb_true_pred = 0

smile_nb_test = 0
emotion_nb_test = 0
gender_nb_test = 0
age_nb_test = 0

for batch_image, batch_label, batch_index in test_data.gen():
  for i in range(len(batch_index)):
    if batch_index[i] == 1.0:
      smile_nb_test += 1
    elif batch_index[i] == 2.0:
      emotion_nb_test += 1
    elif batch_index[i] == 3.0:
      gender_nb_test += 1
    else:
      age_nb_test += 1
  
  feed_dict = {test_model.input_images: batch_image,
               test_model.input_labels: batch_label,
               test_model.input_indexes: batch_index,
               test_model.phase_train: True,
               test_model.keep_prob: 0.5}
  smile_nb_true_pred += test_model.sess.run(test_model.smile_true_pred, feed_dict=feed_dict)
  emotion_nb_true_pred += test_model.sess.run(test_model.emotion_true_pred, feed_dict=feed_dict)
  gender_nb_true_pred += test_model.sess.run(test_model.gender_true_pred, feed_dict=feed_dict)
  age_nb_true_pred += test_model.sess.run(test_model.age_true_pred, feed_dict=feed_dict)

smile_public_test_accuracy = smile_nb_true_pred * 1.0 / smile_nb_test
emotion_public_test_accuracy = emotion_nb_true_pred * 1.0 / emotion_nb_test
gender_public_test_accuracy = gender_nb_true_pred * 1.0 / gender_nb_test
age_public_test_accuracy = age_nb_true_pred * 1.0 / age_nb_test

print('\n')
print('Smile task public test accuracy: ' + str(smile_public_test_accuracy * 100))
print('Emotion task public test accuracy: ' + str(emotion_public_test_accuracy * 100))
print('Gender task public test accuracy: ' + str(gender_public_test_accuracy * 100))
print('Age task public test accuracy: ' + str(age_public_test_accuracy * 100))



Smile task public test accuracy: 91.8
Emotion task public test accuracy: 56.478127612148235
Gender task public test accuracy: 91.6221466731423
Age task public test accuracy: 31.335542429044803
